# Testing the stanford corenlp library

* The library should be downloaded and stored at '{PROJECT_ROOT}/stanford_corenlp_library'
* Using python wrappers at 'https://github.com/Lynten/stanford-corenlp'
* Java needs to be installed to run it.
* Can run on linux without modifications, on Mac something may have to be modified. 


In [5]:
# Necessary imports
from stanfordcorenlp import StanfordCoreNLP
import sys
sys.path.append("../..") #to add the root project directory to the python modules path, so that subdirectories of it can be imported
from src.preparation.data_loading import read_dossier, read_news_article

## Testing it using news articles

In [6]:
# testing it using a news article from the web
airplane_story = 'https://edition.cnn.com/travel/article/best-way-disembark-airplane/index.html'
trump_election_story = 'https://www.nytimes.com/2016/11/09/us/politics/hillary-clinton-donald-trump-president.html'

article_text = read_news_article.read_news_article(trump_election_story)


In [10]:
nlp = StanfordCoreNLP('../../stanford_corenlp_library')
props={'annotators': 'coref','pipelineLanguage':'en','outputFormat':'text'}
results = nlp.annotate('This is a cat. It is very big.')
nlp.close()
results

'{\n  "sentences": [\n    {\n      "index": 0,\n      "parse": "(ROOT\\n  (S\\n    (NP (DT This))\\n    (VP (VBZ is)\\n      (NP (DT a) (NN cat)))\\n    (. .)))",\n      "basicDependencies": [\n        {\n          "dep": "ROOT",\n          "governor": 0,\n          "governorGloss": "ROOT",\n          "dependent": 4,\n          "dependentGloss": "cat"\n        },\n        {\n          "dep": "nsubj",\n          "governor": 4,\n          "governorGloss": "cat",\n          "dependent": 1,\n          "dependentGloss": "This"\n        },\n        {\n          "dep": "cop",\n          "governor": 4,\n          "governorGloss": "cat",\n          "dependent": 2,\n          "dependentGloss": "is"\n        },\n        {\n          "dep": "det",\n          "governor": 4,\n          "governorGloss": "cat",\n          "dependent": 3,\n          "dependentGloss": "a"\n        },\n        {\n          "dep": "punct",\n          "governor": 4,\n          "governorGloss": "cat",\n          "dependent"

# Evaluation

* It runs VERY slowly, it timed before it could even finish running on the newspaper article, it takes a few minutes to even run on 1 sentence...
* The Stanford NLP server also requires a lot of memory to run, a few gigs in practice.
* I think that this library might be too impractical to run in practice.